MILESTONE 2

Week 3-4: TensorFlow Model Architecture and Training Module

IMPORT NECESSARY LIBRARIES

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier  # Importing k-NN
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler


LOADING PREPROCESSED DATASET

In [ ]:


file_path = r"/content/Final_Dataset.csv"

try:
    data = pd.read_csv(file_path)
    print("Data loaded successfully.")
    print(data.head())  # Display the first few rows of the dataset
except FileNotFoundError:
    print("File not found. Please check the file path.")
except Exception as e:
    print(f"An error occurred: {e}")


Data loaded successfully.
       type    amount  oldbalanceOrg  newbalanceOrig   isFraud
0   payment  0.000984       0.013158        0.012321     Fraud
1   payment  0.000186       0.001643        0.001490     Fraud
2  transfer  0.000018       0.000014        0.000000  No Fraud
3  cash_out  0.000018       0.000014        0.000000  No Fraud
4   payment  0.001167       0.003214        0.002297     Fraud


Split the Dataset into Training, Validation, and Testing Sets:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Use train_test_split() from Scikit-learn to divide the data into three sets.
Typical split ratios: 70% for training, 15% for validation, 15% for testing.

In [ ]:

# Load the dataset
file_path = r"/content/Final_Dataset.csv"
data = pd.read_csv(file_path)

# Define features and target variable
X = data.drop('isFraud', axis=1)
y = data['isFraud']

# First split: 70% for training, 30% for temporary (validation + testing)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Second split: Split the 30% temporary set into 15% validation and 15% testing
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Check the sizes of each set
print(f"Training set size: {len(X_train)} samples")
print(f"Validation set size: {len(X_val)} samples")
print(f"Testing set size: {len(X_test)} samples")


Training set size: 9969 samples
Validation set size: 2136 samples
Testing set size: 2137 samples


TRAIN AND EVALUTAION OF DIFFERENT MACHINE LEARNING MODELS

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler

# Load dataset
data = pd.read_csv('/content/Final_Dataset.csv')

# Remove rows with missing values
data = data.dropna()

# Convert categorical variables to numerical (one-hot encoding)
data = pd.get_dummies(data, columns=['type'], drop_first=True)

# Separate features and target variable
X = data.drop('isFraud', axis=1)  # Dropping the target column
y = data['isFraud']  # Setting the target column

# Normalize/Standardize feature values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split dataset into training, validation, and testing sets (70%, 15%, 15%)
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Initialize models
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000),
    "K-Nearest Neighbors": KNeighborsClassifier()
}

# Function to train and evaluate models
def evaluate_models(models, X_train, y_train, X_val, y_val):
    results = {}
    for model_name, model in models.items():
        # Train the model
        model.fit(X_train, y_train)

        # Predict on validation set
        y_pred = model.predict(X_val)

        # Calculate metrics
        accuracy = accuracy_score(y_val, y_pred)
        precision = precision_score(y_val, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_val, y_pred, average='weighted', zero_division=0)

        # Store the results
        results[model_name] = {
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall
        }

    return results

# Evaluate models
evaluation_results = evaluate_models(models, X_train, y_train, X_val, y_val)

# Test set evaluation for all models
test_results = {}
for model_name, model in models.items():
    # Predict on test set
    y_test_pred = model.predict(X_test)

    # Calculate metrics
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred, average='weighted', zero_division=0)
    test_recall = recall_score(y_test, y_test_pred, average='weighted', zero_division=0)

    # Store the results
    test_results[model_name] = {
        'Test Accuracy': test_accuracy,
        'Test Precision': test_precision,
        'Test Recall': test_recall
    }

# Print test set evaluation results for all models
for model_name, metrics in test_results.items():
    print(f"{model_name}:")
    print(f"Accuracy: {metrics['Test Accuracy']:.4f}")
    print(f"Precision: {metrics['Test Precision']:.4f}")
    print(f"Recall: {metrics['Test Recall']:.4f}")
    print("----------------------------------------------------")




Decision Tree:
Accuracy: 0.9972
Precision: 0.9970
Recall: 0.9972
----------------------------------------------------
Random Forest:
Accuracy: 0.9972
Precision: 0.9972
Recall: 0.9972
----------------------------------------------------
Logistic Regression:
Accuracy: 0.9944
Precision: 0.9888
Recall: 0.9944
----------------------------------------------------
K-Nearest Neighbors:
Accuracy: 0.9949
Precision: 0.9939
Recall: 0.9949
----------------------------------------------------


HYPERPARAMETER TUNING FOR RANDOM FOREST MODEL

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler

# Load dataset
data = pd.read_csv('/content/Final_Dataset.csv')

# Remove rows with missing values
data = data.dropna()

# Drop unnecessary columns
# data = data.drop(columns=['nameOrig', 'nameDest'])

# Convert categorical variables to numerical (one-hot encoding)
data = pd.get_dummies(data, columns=['type'], drop_first=True)

# Separate features and target variable
X = data.drop('isFraud', axis=1)  # Dropping the target column
y = data['isFraud']  # Setting the target column

# Normalize/Standardize feature values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split dataset into training, validation, and testing sets (70%, 15%, 15%)
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Initialize the Random Forest model
rf = RandomForestClassifier(random_state=42)

# Train the model before tuning
rf.fit(X_train, y_train)

# Predict on the validation set before tuning
y_val_pred_before = rf.predict(X_val)

# Calculate accuracy before tuning
accuracy_before = accuracy_score(y_val, y_val_pred_before)

# Hyperparameter tuning for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                           scoring='accuracy', cv=3, verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best Random Forest model after hyperparameter tuning
best_rf = grid_search.best_estimator_

# Predict on the validation set after tuning
y_val_pred_after = best_rf.predict(X_val)

# Calculate accuracy after tuning
accuracy_after = accuracy_score(y_val, y_val_pred_after)

# Print only accuracy and precision after tuning
test_accuracy = accuracy_after
test_precision = precision_score(y_val, y_val_pred_after, average='weighted', zero_division=1)

print(f"Accuracy after tuning: {test_accuracy:.4f}")
print(f"Precision after tuning: {test_precision:.4f}")


Fitting 3 folds for each of 108 candidates, totalling 324 fits
Accuracy after tuning: 0.9972
Precision after tuning: 0.9976


Implement the Model:

Use TensorFlow or Keras to create the neural network model.

In [36]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow import keras
from tensorflow.keras import layers

# Step 1: Load the Dataset
data = pd.read_csv('/content/Final_Dataset.csv')  # Update with your dataset path

# Define the label column
label_column = 'isFraud'  # Ensure this matches your actual label column name

# Check if the label column exists
if label_column not in data.columns:
    raise ValueError(f"Label column '{label_column}' not found in the dataset.")

# Identify features and labels
X = data.drop([label_column], axis=1)  # Features

# Convert labels to binary: 'Fraud' -> 1 and any other -> 0
y = data[label_column].apply(lambda x: 1 if x == 'Fraud' else 0).astype(int)

# Define categorical and numerical features
categorical_features = ['type']  # Update with your categorical feature(s)
numerical_features = ['amount', 'oldbalanceOrg', 'newbalanceOrig']  # Update with your numerical feature(s)

# Create a preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ]
)

# Preprocess the features
X_processed = preprocessor.fit_transform(X)

# Step 2: Build the Neural Network Model
model = keras.Sequential([
    layers.Input(shape=(X_processed.shape[1],)),  # Input layer
    layers.Dense(64, activation='relu'),  # Hidden layer
    layers.Dense(32, activation='relu'),  # Hidden layer
    layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                     │ (None, 64)                  │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,625 (10.25 KB)

 Trainable params: 2,625 (10.25 KB)

 Non-trainable params: 0 (0.00 B)

Train the Model with the Training Dataset:

Use the training data to fit the model and monitor its performance on the
validation set.

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming you have run the previous code and have `X_processed` and `y` available

# Step 1: Split the Data
X_train, X_val, y_train, y_val = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Step 2: Train the Model with Early Stopping
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Train the model
history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping])

# Step 3: Evaluate the Model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')


Epoch 1/100
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9213 - loss: 0.2460 - val_accuracy: 0.9951 - val_loss: 0.0385
Epoch 2/100
357/357 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9958 - loss: 0.0292 - val_accuracy: 0.9951 - val_loss: 0.0253
Epoch 3/100
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9937 - loss: 0.0289 - val_accuracy: 0.9954 - val_loss: 0.0208
Epoch 4/100
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9944 - loss: 0.0250 - val_accuracy: 0.9954 - val_loss: 0.0194
Epoch 5/100
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9949 - loss: 0.0215 - val_accuracy: 0.9954 - val_loss: 0.0194
Epoch 6/100
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9933 - loss: 0.0269 - val_accuracy: 0.9954 - val_loss: 0.0195
Epoch 7/100
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9944 - loss: 0.0214 - val_accuracy: 0.9958 - val_loss: 0.0188
Epoch 8/100
357/357 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9956 - loss: 0.0179 - val_accu

CHECKING TRANSCATION IS FRAUD OR NOT

In [63]:
def predict_payment_fraud(input_data):
    """Predict if the payment is fraudulent."""

    # Create a DataFrame from the input data
    input_df = pd.DataFrame([input_data])

    # Preprocess the input data
    input_processed = preprocessor.transform(input_df)

    # Ensure the input_processed is a DataFrame for consistency
    # Get feature names from the preprocessor for the transformed data
    feature_names = preprocessor.get_feature_names_out()

    # Convert to DataFrame
    input_processed_df = pd.DataFrame(input_processed, columns=feature_names)

    # Make prediction
    prediction = model.predict(input_processed_df)
    return 'Fraud' if prediction[0] == 1 else 'No Fraud'


In [65]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import pickle

# Step 1: Load the Dataset
data = pd.read_csv('/content/Final_Dataset.csv')  # Update with your dataset path

# Step 2: Preprocess the Data
label_column = 'isFraud'  # Update with your actual label column name

# Check if the label column exists
if label_column not in data.columns:
    raise ValueError(f"Label column '{label_column}' not found in the dataset.")

X = data.drop([label_column], axis=1)  # Features

# Convert the label column to binary integers (0 or 1)
y = data[label_column].map({'Fraud': 1, 'No Fraud': 0}).astype(int)

# Define categorical and numerical features
categorical_features = ['type']
numerical_features = ['amount', 'oldbalanceOrg', 'newbalanceOrig']

# Create a preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', drop='first'), categorical_features)
    ]
)

# Step 3: Split the Data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Create and Train the Random Forest Model
model = Pipeline(steps=[('preprocessor', preprocessor),
                         ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))])

# Fit the model
model.fit(X_train, y_train)

# Step 5: Save the Model and Preprocessor
with open('random_forest_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

with open('preprocessor.pkl', 'wb') as preprocessor_file:
    pickle.dump(preprocessor, preprocessor_file)

print("Model and preprocessor saved.")

# Step 6: Define a function to predict fraud
expected_categories = ['cash_out', 'debit', 'payment', 'transfer']  # Expected categories for input

def predict_payment_fraud(input_data):
    """Predict if the payment is fraudulent."""

    # Create a DataFrame from the input data
    input_df = pd.DataFrame([input_data])

    # Standardize the input type
    input_df['type'] = input_df['type'].str.strip().str.lower()

    # Check if input type is valid
    if input_df['type'].values[0] not in expected_categories:
        print(f"Warning: Unknown category '{input_df['type'].values[0]}' detected. This will be encoded as all zeros.")

    # Preprocess the input data
    input_processed = preprocessor.transform(input_df)

    # Make prediction
    prediction = model.predict(input_processed)
    return 'Fraud' if prediction[0] == 1 else 'No Fraud'

# Step 7: User Input for Prediction
while True:
    # User inputs for a transaction
    print("Enter the transaction details:")

    transaction_type = input(f"Type ({', '.join(expected_categories)}): ")
    amount = input("Amount: ")
    old_balance = input("Old Balance: ")
    new_balance = input("New Balance: ")

    try:
        amount = float(amount)
        old_balance = float(old_balance)
        new_balance = float(new_balance)

        # Ensure the amount and balances are non-negative
        if amount < 0 or old_balance < 0 or new_balance < 0:
            print("Amounts and balances must be non-negative.")
            continue

        # Create a dictionary to hold the transaction details
        user_input = {
            'type': transaction_type,
            'amount': amount,
            'oldbalanceOrg': old_balance,
            'newbalanceOrig': new_balance
        }

        # Get the prediction
        result = predict_payment_fraud(user_input)
        print(f"The transaction is predicted to be: {result}")

    except ValueError:
        print("Please enter valid numeric values for amount and balances.")

    # Ask if the user wants to check another transaction
    another = input("Do you want to check another trans5action? (yes/no): ")
    if another.lower() != 'yes':
        break


Model and preprocessor saved.
Enter the transaction details:
Type (cash_out, debit, payment, transfer): DEBIT
Amount: 700.0
Old Balance: 800.0
New Balance: 100.0
Please enter valid numeric values for amount and balances.
Do you want to check another trans5action? (yes/no): YES
Enter the transaction details:
Type (cash_out, debit, payment, transfer): TRANSFER
Amount: 4.5
Old Balance: 6.5
New Balance: 2.0
Please enter valid numeric values for amount and balances.
Do you want to check another trans5action? (yes/no): NO
